In [15]:
import csv
import json
import re
import time
import operator

In [16]:
api_key = ""  # from custom search engine API

In [17]:
from googleapiclient.discovery import build

In [18]:
resource = build("customsearch", 'v1', developerKey=api_key).cse()

In [19]:
## list will be ordered after bank name 
unordered = csv.reader(open('AllDataWithWrongBankname.csv'),delimiter=',')
sortedList = sorted(unordered, key=operator.itemgetter(1))



with open("OrderedTestBankenMit?.csv", "w") as sortedfile:
    fileWriter = csv.writer(sortedfile)
    replacement = {"\uFFFD": "?"}


    for row in sortedList:
        string = row[1]
        for key,value in replacement.items():
            insistsInWord = key in string
            if insistsInWord:
                row[1] = re.sub(key, value, string)
            fileWriter.writerow(row)

sortedfile.close()
        


In [1]:
def removeQuestionMarkAtBeginning(existingSubBankNameLeft):
    existingSubBankNameWithChar = re.search("[?]", existingSubBankNameLeft) # wozu?
    existingSubBankNameLeft = re.sub("[?]", '', existingSubBankNameLeft)
    return existingSubBankNameLeft

def changeToUpperCaseLetterAtBeginning(wantedString):
    upper = "{}"
    print("i am in upper case")
    wantedString = upper.format(wantedString)
    return wantedString.capitalize()

    
def makeRequest(actualBankName):
    response = resource.list(q=actualBankName, cx='012204382516046368744:xueylrjvbzs').execute()
    time.sleep(30)
    return response


def loadJSONToString(response):
    json_element = json.dumps(response)  #json object to element 
    json_string = json.loads(json_element) # load json to string
    return json_string
            

def mergePartsToOne(existingSubBankNameLeft, wantedString, existingSubBankNameRight):
    correctedBankName = ""

    if existingSubBankNameLeft != "" :
        correctedBankName += existingSubBankNameLeft

    correctedBankName += wantedString

    if existingSubBankNameRight != "" :
        correctedBankName += existingSubBankNameRight

    print("Correct Bank Name: " + correctedBankName)
    print()
    
    return correctedBankName
    
def extractPartsAfterWantedString(filteredAreaOfWantedString):
    right = "</i></b>"
    existingSubBankNameRight = ""

    # if something is behind the tags
    if len(filteredAreaOfWantedString) > (filteredAreaOfWantedString.index(right)+len(right)) :
        existingSubBankNameRight = filteredAreaOfWantedString[filteredAreaOfWantedString.index(right)+
                                                              len(right):len(filteredAreaOfWantedString)]
        print("string right of searched one = " + existingSubBankNameRight)
    
    return existingSubBankNameRight


def extractPartsBeforeWantedString(filteredAreaOfWantedString):
    left = "<b><i>"
    existingSubBankNameLeft = ""

    # if something is before the tags
    if filteredAreaOfWantedString.index(left) > 0 :  
        existingSubBankNameLeft = filteredAreaOfWantedString[filteredAreaOfWantedString.index(""):
                                                             filteredAreaOfWantedString.index(left)]
        print("string left of searched one = " + existingSubBankNameLeft)
    
    return existingSubBankNameLeft

       

with open("", "r") as uncorrectBanks:
    readCSV = csv.reader(uncorrectBanks)  # statt reader(uncorrectBanks)
    correctedFile = open('CorrectBankNames.csv', 'w')
    
    umlautTuples = ("ä", "ü", "ö", "ß")

    for row in readCSV:
        actualBankName = row[1]
        
        # test if row contains question mark
        extractedBankName = re.search("[?]", actualBankName)
        
        
        if extractedBankName:
            print("wrong formatted name = " + actualBankName)
            
            response = makeRequest(actualBankName)
            
            json_string = loadJSONToString(response)
        
            filteredAreaOfWantedString= ""
    
            if not "spelling" in json_string:
                for umlaut in umlautTuples:
                    potentialBankName = re.sub("[?]", umlaut, actualBankName)
                    response = makeRequest(potentialBankName)
                    json_string = loadJSONToString(response)
                    print(json_string)
                    print("i am in umlaut loop bevor if again with: " + potentialBankName)

                    if "spelling" in json_string :
                        filteredAreaOfWantedString = json_string['spelling']['htmlCorrectedQuery']
                        print ("JSON entry mit " + umlaut + "= " + potentialBankName)
                        umlautTuples = ("") # klappt nicht
                    else:
                        print("Ich bin im else")
            else:
                filteredAreaOfWantedString = json_string['spelling']['htmlCorrectedQuery']
                print ("JSON entry = " + filteredAreaOfWantedString)  
                    
                    
            # Extracting searched string
            right = "</i></b>"
            left = "<b><i>"
            wantedString = filteredAreaOfWantedString[filteredAreaOfWantedString.index(left)+
                                                      len(left):filteredAreaOfWantedString.index(right)]

            if wantedString.islower():
                wantedString = changeToUpperCaseLetterAtBeginning(wantedString)

            print("searched string = " + wantedString)

            # data before wanted string
            subNameLeft = extractPartsBeforeWantedString(filteredAreaOfWantedString)   

            # ? is at the beginning of word
            if re.search("[?]", subNameLeft):
                subNameLeft = removeQuestionMarkAtBeginning(subNameLeft)

            # data after wanted string
            subNameRight = extractPartsAfterWantedString(filteredAreaOfWantedString)


            # override correct name in coloumn
            correctBankName = mergePartsToOne(subNameLeft, wantedString, subNameRight)
            row[1] = correctBankName

            # write data into new file
            writer = csv.writer(correctedFile)
            writer.writerow(row)
        else:
            print("correct bank name: " + actualBankName)
            writer = csv.writer(correctedFile)
            writer.writerow(row)
            print()
            
    correctedFile.close()
    uncorrectBanks.close()
    

FileNotFoundError: [Errno 2] No such file or directory: ''